# Algorithms with Matrices

In this practice we are going to implement algorithms to work with matrices:
- solving simple equations
- inverting matrices
- decomposing matrices to LU decompositions

For these activities Gauss Elimination is our main tool. We will only work with matrices that do not require swapping the rows.

We will also perform QR decomposition using Gramm-Schmidt orthonormalization method

In [1]:
import numpy as np
import lovely_numpy as ln
import json_tricks 
import matplotlib.pyplot as plt

In [2]:
np.random.seed(0)

inputs = json_tricks.load('inputs/inputs.json')
answer = {}

# Task 1. Implement Gauss Elimination Stage I

Your task here is to create an algorithm that will perfrom mutations of matrix $A$ that transform it to Upper-Triangular form $U$.

$$
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1N} \\
a_{21} & a_{22} & \dots & a_{2N} \\
\vdots & \vdots & \ddots & \vdots \\
a_{N1} & a_{N2} & \dots & a_{NN}
\end{bmatrix}

\rightarrow

\begin{bmatrix}
1 & a_{12}^* & \dots & a_{1N}^* \\
0 & 1 & \dots & a_{2N}^* \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
$$

Note that there should be unit diagonal after the 1 stage.

The same mutations for rows as you do to the matrix $A$ should be done to the rows of matrix $B$. 

What can matrix $B$ be? It can be:
- vector of right hand side $b$ (in that case we are solving the System of Linear Equations)
- identity matrix (in that case we invert matrix)
- actually, Stage I can be used as Stage II, and in that case right hand matrix can be some matrix from Stage I


In [3]:
def gauss_elimination(A, B):
    A = A.copy().astype(float)  
    B = B.copy().astype(float)  
    n = len(B)  

    # Forward Elimination
    for i in range(n):
        pivot = A[i, i]
        if abs(pivot) < 1e-8:  
            raise ValueError("Matrix is singular or nearly singular.")
        A[i] = A[i] / pivot
        B[i] = B[i] / pivot

        # Eliminate elements below the pivot
        for j in range(i + 1, n):
            factor = A[j, i]
            A[j] -= factor * A[i]
            B[j] -= factor * B[i]

    # Backward Substitution
    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            B[i] -= A[i, j] * B[j]
            A[i, j] = 0 

    return A, B

In [4]:
answer['gauss_elimination_1'] = [
    gauss_elimination(A=input['A'], B=input['b']) for input in inputs
]

# Task 2: Solve System of Linear Equations (SLE)

For the SLE given by matrix $A$ and right-hand vector $\mathbf b$, implement a function that solves this SLE

You will need to use twice Gauss Elimination Algorithm

1. Apply it to matrix $A$ and to vector $\mathbf b$ as matrix $B$ as Gauss Elimintaion I stage
2. For Gauss Elimination II stage you can use the same algorithm by just applying it to reversed matrix 

    (by enumerating equations and variables in reversed order, we can rewrite the matrix $U$ in low-triangle form $L$). 

    Something has to be done to the right-hand side vector too
3. Form the answer from output of Gauss Elimination Stage II

In [5]:
def solve_equation(A, b):
    res = None

    def reverse_gauss_elimination(A, B):
        n = len(B)
        for i in range(n - 1, -1, -1):
            pivot = A[i, i]
            if abs(pivot) < 1e-8:
                raise ValueError("Matrix is singular or nearly singular.")
            A[i] = A[i] / pivot
            B[i] = B[i] / pivot

            for j in range(i - 1, -1, -1):
                factor = A[j, i]
                A[j] -= factor * A[i]
                B[j] -= factor * B[i]

        return A, B

    U, y = gauss_elimination(A, b)
    L, x = reverse_gauss_elimination(U, y)

    res = x
    return res

You can always check that your solution is correct by subtracting

$A \mathbf x - \mathbf b$, which should give you $\mathbf 0$

By the way, matrix product, such as $A \mathbf x$ in numpy is done using `A @ x` operation

In [6]:
answer['solutions'] = [
    solve_equation(**input) for input in inputs
]

# This should be close to zero
print(inputs[0]['A'] @ answer['solutions'][0] - inputs[0]['b'])

[0. 0. 0. 0. 0. 0.]


# Task 3: Find inverse of $A$

Use the same approach as with solving SLE, but now instead of $\mathbf b$ for right hand side, insert corresponding Identity matrix $I$ as input into Gaussian Elimination algorithm

Identitiy matrix of size $N \times N$ is built using `numpy.eye(N)` 

In [7]:
def find_inverse(A):
    res = None
    n = A.shape[0]
    I = np.eye(n)
    res = gauss_elimination(A, I)[1]
    return res

You can always check that the matrix that you have found is indeed inverse to $A$ as $A^{-1} A = I$

In [8]:
answer['inverses'] = [
    find_inverse(input['A']) for input in inputs
]

# This should be I
print(answer['inverses'][0] @ inputs[0]['A'])

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


# Task 4: LU decomposition

You can find LU decomposition of the matrix $A$ using Gauss Elimination. Again, it is done using 2 stages of Gauss Elimination:

1. $A \mathbf x = I \mathbf b \rightarrow U \mathbf x = L^* \mathbf b$

2. $I U \mathbf x = L^* \mathbf b \rightarrow L U \mathbf x = I \mathbf b$

In [11]:
def find_lu(A):
    L, U = None, None
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy().astype(float)
    for i in range(n):
        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j] -= factor * U[i]
    return L, U

You can always check that your LU decomposition works as

$LU - A = O$ (a zero matrix)

In [12]:
answer['LU'] = [
    find_lu(input['A']) for input in inputs
]

# This should be close to zero
print(answer['LU'][0][0] @ answer['LU'][0][1] - inputs[0]['A'])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# QR decomposition

Now again we will consider matrix $A^T$ in context of SLE:

$A^T \mathbf x = I \mathbf b$

Out of matrix $A$ we can make an orthonormal matrix $Q$ by orthonoramlizing the rows, this process will turn the $I$ matrix on the right to lower-triangular matrix $L^*$:

$A^T \mathbf x = I \mathbf b \rightarrow Q^T \mathbf x = L^* \mathbf b$

After that we should invert matrix $L^*$ and we will get:

$I Q^T \mathbf x = L^* \mathbf b \rightarrow L Q^T \mathbf x = \mathbf b$

If we transpose that matrix, we will get  QR decomposition of matrix $A$:

$A = Q R,$

where $R$ is upper-triangular matrix.

The task is to:
1. Implement Gramm-Schmidt orthonormalization procedure
2. Combining Gramm-Schmidt algorithm and Gaussian Algorithm, implement QR decomposition

# Task 5: Gramm-Schmidt Algorithm for Matrices

Implement Gramm-Schmidt process for **rows** of matrix $A$ (and do the same transforms to matrix $B$)

In [13]:
def gramm_schmidt(A, B):
    A = A.copy().astype(float)
    B = B.copy().astype(float)
    n, m = A.shape
    for i in range(m):
        for j in range(i):
            projection = np.dot(A[:, j], A[:, i]) / (np.dot(A[:, j], A[:, j]) + 1e-8) * A[:, j]
            A[:, i] -= projection
            B[:, i] -= np.dot(B[:, j], A[:, i]) / (np.dot(A[:, j], A[:, j]) + 1e-8)
        
        norm = np.sqrt(np.dot(A[:, i], A[:, i])) + 1e-8
        A[:, i] /= norm
        B[:, i] /= norm
    return A, B

In [14]:
answer['gramm_schmidt'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs
]

print("===== Should be close to 1s: =====")
print((answer['gramm_schmidt'][0][0] ** 2).sum(axis=1))
print()
print("===== Should be close to L: =====")
print(answer['gramm_schmidt'][0][1])

===== Should be close to 1s: =====
[0.02118162 1.33393065 1.0339681  0.99998231 1.08782755 1.52310575]

===== Should be close to L: =====
[[ 7.43294146e-02  4.43966977e-04  1.31986257e-02 -4.93817292e-01
   5.92394859e-01 -1.47782795e+03]
 [ 0.00000000e+00  6.05053871e-02  1.31986257e-02 -4.93817292e-01
   5.92394859e-01 -1.47782795e+03]
 [ 0.00000000e+00  4.43966977e-04  1.17453168e-01 -4.93817292e-01
   5.92394859e-01 -1.47782795e+03]
 [ 0.00000000e+00  4.43966977e-04  1.31986257e-02 -1.22213404e-01
   5.92394859e-01 -1.47782795e+03]
 [ 0.00000000e+00  4.43966977e-04  1.31986257e-02 -4.93817292e-01
   1.01069905e+00 -1.47782795e+03]
 [ 0.00000000e+00  4.43966977e-04  1.31986257e-02 -4.93817292e-01
   5.92394859e-01 -1.27789842e+03]]


# Task 6: QR decomposition

Implement QR decomposition algorithm. Note that we work on rows on matrix $A$ with our Gauss and Gramm-Schmidt algorithms, but QR decomposition is defined for column matrix 

In [17]:
def find_qr(A):
    B = np.eye(A.T.shape[0])
    Q = A.copy().astype(float)
    R = np.zeros_like(A, dtype=float)
    n, m = A.shape

    for i in range(m):
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], Q[:, i])
            Q[:, i] -= R[j, i] * Q[:, j]
        R[i, i] = np.sqrt(np.dot(Q[:, i], Q[:, i])) + 1e-8
        Q[:, i] /= R[i, i]
    return Q, R

In [18]:
answer['QR'] = [
    find_qr(input['A']) for input in inputs
]

print("===== Should be close to O: =====")
print(answer['QR'][0][0] @ answer['QR'][0][1] - inputs[0]['A'])

===== Should be close to O: =====
[[ 0.00000000e+00  0.00000000e+00  7.67773981e-18  0.00000000e+00
   6.66133815e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.44089210e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.77635684e-15
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.42108547e-14 -1.42108547e-14
   7.10542736e-15 -7.10542736e-15]
 [ 0.00000000e+00  0.00000000e+00 -1.42108547e-14 -3.55271368e-15
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.06581410e-14  0.00000000e+00]]


In [ ]:
json_tricks.dump(answer, '.answer.json')